In [20]:
import numpy as np
import pandas as pd
from sklearn.ensemble import IsolationForest
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler 
import seaborn as sns 
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import ElasticNet
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import StackingRegressor

In [27]:
train = pd.read_csv(r"C:\Users\Administrator.DAI-PC2\Downloads\Shubham\Practical Machine Learning\Kaggle\Flood Prediction\train.csv")
test = pd.read_csv(r"C:\Users\Administrator.DAI-PC2\Downloads\Shubham\Practical Machine Learning\Kaggle\Flood Prediction\test.csv")

In [28]:
train.shape

(1117957, 22)

In [29]:
#### Cross val score with 5 fold validation without outliers

In [30]:
df = train.drop(["id"],axis=1)
X= df.drop("FloodProbability",axis=1)
clf = IsolationForest(contamination=0.05, random_state=24)
clf.fit(X)
predictions = clf.predict(X)

print("%age of outliers="+ str((predictions<0).mean()*100)+ "%")
abn_ind = np.where(predictions < 0)
print("Outliers:")
outliers = df.index[abn_ind]
print(outliers)

%age of outliers=5.000013417331794%
Outliers:
Index([     14,      26,      28,      44,      75,      83,     112,     135,
           157,     160,
       ...
       1117689, 1117742, 1117754, 1117803, 1117837, 1117848, 1117854, 1117895,
       1117900, 1117925],
      dtype='int64', length=55898)


In [31]:
df = df.drop(outliers)
X= df.drop("FloodProbability",axis=1)
y = df["FloodProbability"]
X_test= test.drop("id",axis=1)

In [32]:
elastic = ElasticNet()
lr = LinearRegression()
xgb = XGBRegressor(random_state=24)
lgbm = LGBMRegressor(random_state=24)
cat =  CatBoostRegressor(random_state=24)
stack3 = StackingRegressor([('LGBM',lgbm),('Elastic',elastic),('XGB',xgb),('LR',lr)],final_estimator=cat,passthrough=True )

stack3.fit(X,y)

y_pred  = stack3.predict(X_test)
submit = pd.DataFrame({"id":test["id"],"FloodProbability":y_pred})
submit.to_csv(r"C:\Users\Administrator.DAI-PC2\Downloads\Shubham\Practical Machine Learning\Kaggle\Flood Prediction\stack3.csv",index=False)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.074481 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 344
[LightGBM] [Info] Number of data points in the train set: 1062059, number of used features: 20
[LightGBM] [Info] Start training from score 0.503684
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.057910 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 346
[LightGBM] [Info] Number of data points in the train set: 849647, number of used features: 20
[LightGBM] [Info] Start training from score 0.503667
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.058258 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 345
[LightGBM] [Info] Number of data points in the train set: 849647, number of used features: 20
[LightGBM] [Info] Start tr